<a href="https://colab.research.google.com/github/ckj18/ToBigs_workspace/blob/main/vision_adv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from tqdm import tqdm

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms


# 코드 다시 돌리기 위한 seed 고정
import random
import numpy as np
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [2]:
# 3X3 conv layer
def conv(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), 
              nn.BatchNorm2d(out_channels), 
              nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

# ResNet 이용 out = res(x)+x
class MyModel(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(MyModel, self).__init__()
        self.conv1 = conv(in_channels, 64)
        self.conv2 = conv(64, 128, pool=True)
        self.res1 = nn.Sequential(conv(128, 128), 
                                  conv(128, 128))
        
        self.conv3 = conv(128, 256, pool=True)
        self.conv4 = conv(256, 512, pool=True) 
        self.res2 = nn.Sequential(conv(512, 512), 
                                  conv(512, 512)) 

        self.conv5 = conv(512, 1028, pool=True) 
        self.res3 = nn.Sequential(conv(1028, 1028), 
                                  conv(1028, 1028))  
        
        self.fc = nn.Sequential(nn.MaxPool2d(2), 
                                        nn.Flatten(),
                                        nn.Linear(1028, num_classes)) 
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.res1(x) + x

        x = self.conv3(x)
        x = self.conv4(x)
        x = self.res2(x) + x

        x = self.conv5(x)
        x = self.res3(x) + x

        x = self.fc(x)
        return x     
    
model = MyModel(3, 100).to(device)
model

MyModel(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (res1): Sequential(
    (0): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=Tr

In [3]:
# 이미지 RGB 값에 대한 평균과 편차를 측정
def get_mean_std(dataset):
  means = torch.zeros(3)
  stds = torch.zeros(3)

  for img, label in dataset:
      means += torch.mean(img, dim = (1,2))
      stds += torch.std(img, dim = (1,2))

  means /= len(dataset)
  stds /= len(dataset)
    
  print(f'mean: {means}')
  print(f'std: {stds}')

  return means.numpy(), stds.numpy()

In [4]:
# mean과 std를 계산하기 위해 train 다운로드 

train = torchvision.datasets.CIFAR100(root="./", train=True, download=True, transform=transforms.ToTensor())

Files already downloaded and verified


In [5]:
# mean, std 계산
means, stds = get_mean_std(train)

mean: tensor([0.5071, 0.4866, 0.4409])
std: tensor([0.2009, 0.1984, 0.2023])


In [ ]:
# train에 data augmentation 적용(RandomCrop, RandomHorizontalFlip)
# train, test에 Normalize 적용(means, stds)
train_transform = transforms.Compose([   
    transforms.RandomCrop(32, padding=4,padding_mode='reflect') ,
    transforms.RandomHorizontalFlip(), 
    transforms.ToTensor(),
    transforms.Normalize(means, stds, inplace=True)
])        

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(means, stds)
])  

In [6]:
# train, test에 transform 적용
train = torchvision.datasets.CIFAR100(root="./", train=True, download=True, transform=train_transform)
test = torchvision.datasets.CIFAR100(root="./", train=False, download=True, transform=test_transform)

# batch_size=256
train_loader = torch.utils.data.DataLoader(train, batch_size=256,
                                           shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test, batch_size=256,
                                          shuffle=False, num_workers=2)

# optimizer: Adam, learning_rate=1e-3, weight_decay=1e-3 적용
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-3)
criterion = nn.CrossEntropyLoss()

# val_acc가 60%를 안정적으로 도달하기 위한 카운트 수
check_count = 3

Files already downloaded and verified
Files already downloaded and verified


In [7]:
for epoch in range(100):
    model.train()
    running_loss = 0.0
    valid_loss = 0.0
    best_acc = 0.0
    best_model_wts = model.state_dict()
    print(f"train epoch: {epoch+1}----------------")
    for img, label in tqdm(train_loader):
        img = img.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        output = model(img)
        loss = criterion(output, label)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    correct, all_data = 0,0
    print("\ntrain_loss : ", running_loss / len(train_loader))
    model.eval()
    for img, label in test_loader:
        with torch.no_grad():
            img = img.to(device)
            label = label.to(device)
            output = model(img)
            loss = criterion(output, label)
            valid_loss += loss.item()

            correct += torch.sum(torch.argmax(output, dim=1) == label).item()
            all_data += len(label)
    print("val_loss : ", valid_loss / len(test_loader))
    print("val_acc : ", correct / all_data)
    if correct / all_data > best_acc:
        best_acc = correct / all_data
        best_model_wts = model.state_dict()

    # 60%를 도달할 때마다 카운트 감소, 0이되면 학습 조기 종료
    if correct / all_data >= 0.6:
        check_count -= 1
        if check_count == 0:
          break
    

train epoch: 1----------------


100%|██████████| 196/196 [00:44<00:00,  4.38it/s]


train_loss :  4.2684140302697005


val_loss :  3.7288798093795776
val_acc :  0.1423
train epoch: 2----------------


100%|██████████| 196/196 [00:43<00:00,  4.49it/s]


...


train_loss :  0.9880453962452558


val_loss :  1.374435120820999
val_acc :  0.6199
train epoch: 17----------------


100%|██████████| 196/196 [00:39<00:00,  4.94it/s]


train_loss :  0.9568425961294953


val_loss :  1.4584486305713653
val_acc :  0.6009
